# AWS 환경 설정 및 자동자막에 필요한 모듈 정리

AWS에서 Transcribe를 로컬로 사용하기 위한 환경설정과 자동 자막에 사용하는 모듈을 정리했다. 목차는 아래와 같다.

1. Python과 AWS 연동하기
2. 자동 자막화 모듈 코드

## 1. Python과 AWS 연동하기
transcribe 서비스는 기본적으로 AWS 콘솔에 접속해 사용할 수 있다. AWS는 요금정책이 철저하고 보안이 강력해서 해당 서비스를 개인 pc 환경에서 구동하려면 AWS CLI(AWS Command Libe Interface)를 설치해야 한다. 해당 링크를 통해 [AWS CLI를 설치](https://docs.aws.amazon.com/ko_kr/cli/latest/userguide/cli-chap-install.html)하고 잘 설치되었는지 확인한다. 설치 되었는지를 확인하기 위해서 명령 프롬프트에 `aws --version`명령어를 입력해 설치된 버전을 확인할 수 있다.

<b>*참고: CLI버전은 두 가지 버전이 있다. 메이저하게 사용하는 버전은 버전 2이므로 특별한 경우가 아니라면 2를 설치한다.</b>

In [1]:
!aws --version

aws-cli/2.0.14 Python/3.7.7 Windows/10 botocore/2.0.0dev18


[AWS IAM](https://aws.amazon.com/ko/iam/)으로 권한 설정을 마친 사용자를 등록한다. 해당 계정은 최소한의 권한만을 허용하기 위해 AmazonS3FullAccess와 AmazonTranscribeFullAccess 권한을 부여했다. 사용자 구성을 완료하면 엑세스 키를 발급해야 한다. 엑세스 키를 발급하려면 [구성 기본 사항](https://docs.aws.amazon.com/ko_kr/cli/latest/userguide/cli-configure-quickstart.html#cli-configure-quickstart-creds) 문서의 <b>엑세스 키 및 보안 엑세스 키</b> 부분에 적힌 절차를 순서대로 따라한다.

PC의 명령 프롬프트에 `aws configure` 명령어를 실행해 사용자를 구성한다. 사용자 구성 시, 리전을 아시아 태평양(서울)로 설정하려면 `ap-northeast-2`를 적으면 된다. output format은 json으로 설정하고 AWS에서 발급한 Secret Key를 입력한 뒤 구성을 마친다. 사용자 구성에 대한 자세한 설명은 [링크](https://docs.aws.amazon.com/ko_kr/cli/latest/userguide/cli-configure-quickstart.html)에서 확인한다.

사용자 구성을 마쳤다면 [boto3을 설치](https://github.com/boto/boto3)한다. boto3은 로컬 PC에서 python으로 AWS 서버에 명령을 주고 받기 위한 모듈이 모여있는 개발 키트이다. boto3이 정상적으로 설치되었는지를 확인하기 위해 AWS SDK를 사용하는 예제를 실행한다.

<b>*AWS S3에 만들어진 버켓이 아무것도 없다면 아무것도 출력되지 않고 정상종료된다.</b>

In [3]:
import boto3

s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

## 2. 자동자막화 모듈 코드

자동자막화에 필요한 모듈은 아래와 같다.
1. AWS S3에 파일 업로드
2. AWS  transcribe 수행 후 자막 파일 다른 이름으로 저장

### 2-1. AWS S3에 파일 업로드

AWS에서 제공하는 클라우드 스토리지 서비스(S3)에 로컬 지정 파일을 업로드하고 `url`변수에 해당 파일의 주소를 리턴한다. 이 동작을 수행하기 위해서는 아래의 선행조건이 갖춰져야 한다.

- S3에 CLI로 등록한 사용자가 사용하는 버켓이 있고 그 이름을 알고있다.
- IAM 사용자가 S3에 Access할 수 있는 권한이 있다.

AWS의 S3 Management Conlsole에 접속해 <b>+버킷 만들기</b>버튼을 클릭해 버킷을 만들 수 있다. 버킷은 DNS형식으로 접근하게 되므로 버킷의 이름은 다른 사용자가 만든 버킷 이름과 겹치면 안된다. 옵션 및 권한은 기본으로 설정해주는 것을 그대로 따라 설정해도 된다.

아래의 코드는 S3의 버켓에 파일을 업로드하는 코드이다.

In [ ]:
# S3에 파일 업로드하기
import boto3

# S3 클라이언트 생성
s3 = boto3.client('s3')

# 업로드할 파일의 이름
filename = '업로드할_파일.mp3'

# 파일을 담을 S3 버킷 이름
bucket_name = '버켓이름'

#파일을 S3에 업로드한다.
s3.upload_file(filename, bucket_name, filename)

#업로드한 파일의 url을 얻는다.
location = s3.get_bucket_location(Bucket=bucket_name)['LocationConstraint']
url = "https://s3-%s.amazonaws.com/%s/%s" % (location, bucket_name, filename)

`filename = 'audio/transcribe-sample.mp3'`처럼 폴더 내의 파일을 업로드하면 `audio`폴더도 업로드된다. 폴더가 통으로 올라가면 경로 문제가 발생할 수 있으므로 이 python코드와 업로드할 파일을 같은 폴더에 넣자.

### 2-2. AWS Transcribe 수행 후 자막 파일 다른 이름으로 저장

이 코드의 동작 순서는 아래와 같다.

1. transcribe job을 만들고 json파일이 완성될 때까지 로딩 메시지를 출력한다.
2. json파일이 완성되면 다른 이름으로 저장하는 창이 뜬다.
3. transcribe job과 S3에 업로드된 파일을 모두 삭제한다.

transcribe에 올라온 job을 삭제하는 이유는 비용 절감도 있지만, <b>job의 이름을 겹치지 않게 하기 위함</b>도 있다. job의 이름이 겹치면 transcribe 서비스가 시작되지 않기 때문에 주의가 필요하다.

이 동작을 수행하기 위해서는 아래의 선행조건이 갖춰져야 한다.

- S3에 transcribe를 수행할 수 있는 파일(음성, 영상 파일)이 업로드된 상태고, 해당 파일의 url을 알고 있다.
- IAM 사용자가 transcribe에 접근할 권한이 있다.

In [ ]:
from __future__ import print_function
import time
import boto3

transcribe = boto3.client('transcribe')
job_name = "sdk-test"
# transcribe를 돌릴 job의 이름이다.

job_uri = url
#aws S3에 업로드한 파일의 url 입력
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri':job_uri},
    MediaFormat='mp3',
    LanguageCode='en-US'
)
# MediaForamt: transcribe를 돌리는 파일의 형식
# LanguageCode: 자막을 생성할 언어를 선택한다. 한국어 코드는 'ko-KR'이다.

while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
    
import webbrowser

url = status["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]

# 결과 json파일을 다운받는 url 오픈. 다른 이름으로 저장 창이 뜸.
webbrowser.open(url)

# transcribe job을 삭제한다.
transcribe.delete_transcription_job(TranscriptionJobName=job_name)
'''
# 업로드한 파일의 이름
filename = '업로드할_파일.mp3'
# 파일을 담은 S3 버킷 이름
bucket_name = '버켓이름'
'''
#s3에 올라온 파일을 삭제한다.
s3.delete_object(Bucket=bucket_name, Key=filename)


업로드한 파일 이름과 S3이름이 담긴 변수 `filename`, `bucket_name`이 이 코드를 실행할 때도 살아있으면 그대로 실행해도 되지만, 없다면 `'''`로 감싸진 부분의 주석을 풀어준다.

<b>*json파일 다운이 끝나고 job이 성공적으로 삭제되는지 확인하지 않았음. 다운이 끝나기도 전에 job이 삭제되는 오류가 발생하면 json파일 다운과 job삭제를 다른 모듈로 분리할 것을 추천.</b>